In [ ]:
import numpy as np
import pandas as pd
from scipy import signal

In [ ]:
fs_bvp = 64  

In [ ]:
lowcut = 0.5  
highcut = 5.0  
order = 3 

nyquist = fs_bvp / 2
b, a = signal.butter(order, [lowcut / nyquist, highcut / nyquist], btype="band")
bvp_filtered = signal.filtfilt(b, a, bvp_raw)

In [ ]:
min_distance = int(fs_bvp * 0.4)  
peaks, _ = signal.find_peaks(bvp_filtered, distance=min_distance)

In [ ]:
ibi = np.diff(t[peaks]) 
hr_instantaneous = 60.0 / ibi


In [ ]:
valid_mask = (hr_instantaneous >= 40) & (hr_instantaneous <= 200)
hr_valid = hr_instantaneous[valid_mask]
ibi_valid = ibi[valid_mask]
hr_time = [peaks][1:][valid_mask] 

In [ ]:
window_size = 30 
step_size = 1  

hr_smooth_time = np.arange(hr_time[0], hr_time[-1], step_size)
hr_smooth = []

for t0 in hr_smooth_time:
    in_window = (hr_time >= t0) & (hr_time < t0 + window_size)
    if np.any(in_window):
        hr_smooth.append(np.mean(hr_valid[in_window]))
    else:
        hr_smooth.append(np.nan)

hr_smooth = np.array(hr_smooth)

valid_idx = ~np.isnan(hr_smooth)
hr_final = hr_smooth[valid_idx]
hr_final_time = hr_smooth_time[valid_idx]